In [1]:
from google.colab import files
uploaded = files.upload()

Saving cleaned_logs.csv to cleaned_logs.csv


In [2]:
import pandas as pd
import numpy as np
from urllib.parse import urlparse

In [4]:
df = pd.read_csv("cleaned_logs.csv")
df.head()

,ip,time,hour,method,url,status,size,is_bot
0,54.36.149.41,2019-01-22 03:56:14+03:30,3,GET,/filter/27|13%20%D9%85%DA%AF%D8%A7%D9%BE%DB%8C...,200,30577,1
1,31.56.96.51,2019-01-22 03:56:16+03:30,3,GET,/image/60844/productModel/200x200,200,5667,0
2,31.56.96.51,2019-01-22 03:56:16+03:30,3,GET,/image/61474/productModel/200x200,200,5379,0
3,40.77.167.129,2019-01-22 03:56:17+03:30,3,GET,/image/14925/productModel/100x100,200,1696,1
4,91.99.72.15,2019-01-22 03:56:17+03:30,3,GET,/product/31893/62100/%D8%B3%D8%B4%D9%88%D8%A7%...,200,41483,0


In [5]:
def url_features(u):
    parsed = urlparse(u)
    path = parsed.path.lower()

    return pd.Series({
        "url_length": len(u),
        "num_slashes": path.count("/"),
        "has_query": 1 if parsed.query else 0,
        "is_static": 1 if any(ext in path for ext in [".png", ".jpg", ".css", ".js", ".gif"]) else 0,
        "is_api": 1 if any(x in path for x in ["filter", "update", "api"]) else 0
    })

df[["url_length","num_slashes","has_query","is_static","is_api"]] = df["url"].apply(url_features)

In [6]:
df["is_post"] = (df["method"] == "POST").astype(int)

In [7]:
df["is_error"] = (df["status"] != 200).astype(int)

In [9]:
df["is_night"] = (df["hour"] < 6).astype(int)

In [10]:
ipgrp = df.groupby("ip")

df["requests_per_ip"] = df["ip"].map(ipgrp.size())
df["unique_urls_per_ip"] = df["ip"].map(ipgrp["url"].nunique())
df["error_rate_per_ip"] = df["ip"].map(ipgrp["is_error"].mean())
df["avg_size_per_ip"] = df["ip"].map(ipgrp["size"].mean())

In [13]:
ip_bot_ratio = df.groupby('ip')['is_bot'].mean()
df['bot_ratio_per_ip'] = df['ip'].map(ip_bot_ratio)

In [14]:
features = df[[
    "url_length",
    "num_slashes",
    "has_query",
    "is_static",
    "is_api",
    "is_post",
    "is_error",
    "hour",
    "is_night",
    "requests_per_ip",
    "unique_urls_per_ip",
    "error_rate_per_ip",
    "avg_size_per_ip",
    "bot_ratio_per_ip",
    "size"
]]

features.head()

,url_length,num_slashes,has_query,is_static,is_api,is_post,is_error,hour,is_night,requests_per_ip,unique_urls_per_ip,error_rate_per_ip,avg_size_per_ip,bot_ratio_per_ip,size
0,166,2,0,0,1,0,0,3,1,6,6,0.000000,31859.500000,1.0,30577
1,33,4,0,0,0,0,0,3,1,78,72,0.000000,10989.923077,0.0,5667
2,33,4,0,0,0,0,0,3,1,78,72,0.000000,10989.923077,0.0,5379
3,33,4,0,0,0,0,0,3,1,14,14,0.000000,10484.785714,1.0,1696
4,146,4,0,0,0,0,0,3,1,3299,3144,0.001819,41566.940588,0.0,41483


In [15]:
features.to_csv("processed_features.csv", index=False)